In [409]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

# Load data

In [410]:
df = pd.read_csv("Dry_Bean_Dataset.csv")

In [411]:
df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,roundnes,Class
0,114004,1279.356,451.361256,323.747996,0.875280,BOMBAY
1,117034,1265.926,425.923788,351.215109,0.917710,BOMBAY
2,126503,1326.959,475.772459,339.381887,0.902809,BOMBAY
3,128118,1360.135,504.024964,NaN,0.870274,BOMBAY
4,129409,1348.888,484.364424,341.172659,0.893763,BOMBAY
...,...,...,...,...,...,...
145,35476,711.317,266.850827,169.928349,0.881086,SIRA
146,35506,713.086,262.106215,173.247828,0.877461,SIRA
147,35561,708.141,259.768348,174.810552,0.891137,SIRA
148,35570,710.530,263.260383,173.378732,0.885378,SIRA


In [412]:
import numpy as np
from sklearn.utils import shuffle


def preprocess_data(df, selected_features, selected_classes,Activation="sigmoid"):
    # Filter data based on selected classes
    df_filtered = df[df['Class'].isin(selected_classes)]

    # Handle null values
    for feature in selected_features:
        df_filtered.loc[:, feature] = df_filtered[feature].fillna(df_filtered[feature].mean())
    
    # Encoding and dropping columns
    # if Activation=="sigmoid":
    #     df_filtered['Class_encoded'] = np.where(df_filtered['Class'] == selected_classes[0], 1,0)
    # else:
    #     df_filtered['Class_encoded'] = np.where(df_filtered['Class'] == selected_classes[0], 1, -1)

    # mapping = {'BOMBAY': -1, 'CALI': 0, 'SIRA': 1}
    # df_filtered.loc[:, 'Class_encoded'] = df_filtered['Class'].map(mapping)
    # df_filtered.drop(columns=['Class'], inplace=True)
    
    # Outlier handling
    def find_range(col, df, class_value):
        class_rows = df[df['Class'] == class_value][col]
        Q1, Q3 = class_rows.quantile(0.25), class_rows.quantile(0.75)
        IQR = Q3 - Q1
        lower_range = Q1 - 1.5 * IQR
        upper_range = Q3 + 1.5 * IQR
        return lower_range, upper_range
    
    for col in selected_features:
        for class_value in selected_classes:
            lower_limit, upper_limit = find_range(col, df_filtered, class_value)
            class_rows = df_filtered['Class'] == class_value
            df_filtered.loc[class_rows, col] = df_filtered.loc[class_rows, col].clip(lower_limit, upper_limit)
    
    # Scaling Standard Scalling
    df_filtered.loc[:, selected_features] = (df_filtered[selected_features] - df_filtered[selected_features].mean()) / df_filtered[selected_features].std()
    

    # Splitting
    X = df_filtered[selected_features]
    y = df_filtered["Class"]
    
    return X, y

In [413]:
df.columns

Index(['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'roundnes',
       'Class'],
      dtype='object')

In [414]:
X ,y = preprocess_data(df,['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'roundnes'],['BOMBAY', 'CALI',"SIRA"])

# Splitting into training and testing

In [415]:
one_hot_encoded = pd.get_dummies(y)
one_hot_encoded

,BOMBAY,CALI,SIRA
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
145,0,0,1
146,0,0,1
147,0,0,1
148,0,0,1


In [416]:
X_train, X_test, y_train, y_test = train_test_split(X, one_hot_encoded, test_size=0.2, random_state=42,stratify=y)

# Cost Function

In [417]:
def sigmoid(x):
    x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x))

# FeedForward

In [418]:
input_size = 5
hidden_size = 4
output_size = 3

# Between input & hidden layer
W1 = np.random.randn(hidden_size, input_size)* 0.01
b1 = np.zeros((hidden_size, 1))

# Between hidden layer & output
W2 = np.random.randn(output_size, hidden_size)* 0.01
b2 = np.zeros((output_size, 1))

def forward_propagation(X, W1, b1, W2, b2):
    F1 = np.dot(W1, X.T) + b1
    A1 = sigmoid(F1)
    
    F2 = np.dot(W2, A1) + b2
    A2 = sigmoid(F2)
    
    return A1 , A2

A1_train, _ = forward_propagation(X_train, W1, b1, W2, b2)
A1_test, _ = forward_propagation(X_test, W1, b1, W2, b2)


## Creating a Neural Network from Scratch

### Creating Neuron Class

In [419]:
class Neuron():
    def __init__(self,random_state=42,lr=0.001,activation="sigmoid",biasFlag=True,input=None,output_Neuron=False,Y_Actual=None) -> None:
        np.random.seed(random_state)
        
        self.input = input
        self.activation=activation
        self.learning_rate=lr
        self.weights=None
        self.biasFlag=biasFlag
        self.bias=None
        self.output=0
        self.error=None
        
        self.Y_Actual=Y_Actual
        
        
    
    def init_weights(self,input_size):

        self.weights=np.random.randn(input_size,1)

        if self.biasFlag:
            self.bias = 1
        else:
            self.bias=0

    #Forward Functions
    def linear_forward(self):
        Z= np.dot(self.weights.T,self.input.T)+self.bias
        return Z
    
    def sigmoid(self,Z):
        return 1 / (1 + np.exp(-Z))
    
    def tanh(self,Z):
        return (np.exp(Z)-np.exp(-Z))/(np.exp(Z)+np.exp(-Z))
    
    def activation_forward(self):
        Z = self.linear_forward()
        if self.activation=='sigmoid':

            A = self.sigmoid(Z)

        elif self.activation=='tanh':

            A = self.tanh(Z)
        
        self.output=A

        return A
    
    #Backward Functions


    def sigmoid_backward(self):
        return self.output * (1-self.output)
    
    def tanh_backward(self):
        return 1 - np.power(self.output,2)
    
    def NeuronError(self,NextError=None,OutputLayer=False,weights=None): ## error of next layer
        if self.activation=='sigmoid':

            Drev = self.sigmoid_backward()

        elif self.activation=='tanh':

            Drev = self.tanh_backward()

        if OutputLayer:


            self.error=(self.Y_Actual-self.output)*Drev
            return self.error
        else:
            
            # print(NextError)
            # print(weights.T)
            # print(np.dot(NextError,weights.T))
            self.error=Drev*np.sum(np.dot(NextError,weights.T))
            return self.error
        
        # print(self.error.shape)
    def update_weights(self):
        # print(self.weights)
        # print("Neuron error",self.error.shape)
        # print("Neuron Inputs",self.input.shape)
        # print("Neuron Weights Before Update",self.weights.shape)
        self.weights=self.weights+(self.learning_rate*np.dot(self.error,self.input).T)
        # print("Neuron Weights After",self.weights.shape)


In [420]:
np.array([1,2,5,4]).reshape(1,-1).shape

(1, 4)

In [421]:
# Neuron1=Neuron(input=np.array([1,2,5,4]).reshape(1,-1),Y_Actual=np.array([100000]).reshape(1,-1),output_Neuron=True)
# Neuron1.init_weights(4)
# Neuron1.activation_forward()
# Neuron1.NeuronError(OutputLayer=True)
# Neuron1.update_weights()

### Creating Layer Class

In [422]:
class Layer():
    def __init__(self,random_state=42,lr=0.001,activation="sigmoid",biasFlag=True,layer_input=None,neurons=1) -> None:
        np.random.seed(random_state)
        self.weights=[i for i in range(neurons)]
        self.biasFlag=biasFlag
        self.learning_rate=lr
        self.activation=activation
        self.neurons=[Neuron(random_state=random_state+i,lr=lr,activation=activation,biasFlag=biasFlag,input=layer_input) for i in range(neurons)]
        self.layer_input=layer_input
        self.outputlayer=False
        self.layerOutput=[i for i in range(len(self.neurons))]
        self.layerError=[i for i in range(len(self.neurons))]
    
    def init_weights(self,input_size):
        
        self.weights=[i for i in range(len(self.neurons))]
        for i in range(len(self.neurons)):
            self.neurons[i].init_weights(input_size)
            self.weights[i]=self.neurons[i].weights
        self.weights=np.array(self.weights).reshape(-1,len(self.neurons))
        
    
    #Forward Functions
    
    def forward(self,X=None):
        self.layer_input=X
        self.layerOutput=[i for i in range(len(self.neurons))]
        for i in range(len(self.neurons)):
            self.neurons[i].input=X
            self.layerOutput[i]=self.neurons[i].activation_forward()
        self.layerOutput=np.array(self.layerOutput).reshape(-1,len(self.neurons))
    
    def backward(self,Y_actual=None,error=None,weights=None):
        
        self.layerError=[i for i in range(len(self.neurons))]
        
        for i in range(len(self.neurons)):
            self.neurons[i].Y_Actual=Y_actual
            self.layerError[i]=self.neurons[i].NeuronError(error,self.outputlayer,weights)

        self.layerError=np.array(self.layerError).reshape(-1,len(self.neurons))
        # print("Layer error",self.layerError.shape)
    
    def update_weights(self):
        
        self.weights=[i for i in range(len(self.neurons))]
        
        for i in range(len(self.neurons)):
            
            self.layerError[i]=self.neurons[i].update_weights()
            self.weights[i]=self.neurons[i].weights
        self.weights=np.array(self.weights).reshape(-1,len(self.neurons))
        
    

In [423]:

Layer1 = Layer(42,0.001,"sigmoid",True,np.array([1,2,5,4]).reshape(1,-1),neurons=3)


### Initialize The Network

In [424]:
class Network():

    def __init__(self,num_layers:int,neurons:list,random_state=42,lr=0.001,activation="sigmoid",bias=True,epochs=10) -> None:
        # np.random.seed(random_state)
        if(len(neurons)!=num_layers):
            print("Number of neurons per layer do not match number of layers")
            return
        self.result=None
        self.num_layers=num_layers+1 #output layer
        neurons.append(3) # output layer
        self.layers=[Layer(random_state=random_state,lr=lr,activation=activation,biasFlag=bias,neurons=neurons[i]) for i in range(self.num_layers)]
        self.epochs=epochs
        


    def Train(self,X,y):

        #Initialization
        self.layers[0].init_weights(X.shape[1])
        # print("Layer # 1\n",self.layers[0].weights)

        for i in range(1,self.num_layers):
            self.layers[i].init_weights(len(self.layers[i-1].layerOutput))
            # print(f"Layer # {i+1}\n",self.layers[i].weights)
        
        self.layers[-1].outputlayer=True
        
        #--------------------------------------------------------
    
        for _ in range(self.epochs):

        #Forward
            
            self.layers[0].forward(X) #(5,120)
            
            for i in range(1,self.num_layers):
                
                self.layers[i].forward(self.layers[i-1].layerOutput)
                # print(self.layers[i].layer_input.shape)
                # print(self.layers[i].layerOutput.shape)
            
            self.result=self.layers[-1].layerOutput
        
        #Backward Propagation

            self.layers[-1].backward(np.argmax(y,axis=1))
            
            for i in reversed(range(0,self.num_layers-1)):
                self.layers[i].backward(Y_actual=None,error=self.layers[i+1].layerError,weights=self.layers[i+1].weights)
        
        # Update
            for i in range(self.num_layers):
                self.layers[i].update_weights()

        
    def Test(self,X):
        

        self.layers[0].forward(X) #(5,120)
            
        for i in range(1,self.num_layers):
                
            self.layers[i].forward(self.layers[i-1].layerOutput)
            # print(self.layers[i].layer_input.shape)
            # print(self.layers[i].layerOutput.shape)
        
        self.result=self.layers[-1].layerOutput
        
        return np.argmax(self.layers[-1].layerOutput,axis=1)
    

In [435]:
Network1=Network(1,[3],42,0.001,'sigmoid',True,1000)

Network1.Train(X_train.to_numpy(),y_train.to_numpy())

NetworkOutput=Network1.Test(X_test)

In [436]:
(np.argmax(y_test.to_numpy(),axis=1)==NetworkOutput).sum()/y_test.shape[0]

0.5

In [427]:
# ((np.argmax(y_train.to_numpy(),axis=1))==np.argmax(Network1.result,axis=1)).sum()/y_train.shape[0]

In [428]:
Network1.layers[-1].layerOutput

array([[0.94867594, 0.94850385, 0.9500495 ],
       [0.94833968, 0.94925088, 0.94976858],
       [0.94836262, 0.94966487, 0.94978411],
       [0.9469192 , 0.94589096, 0.94597761],
       [0.94994406, 0.94684279, 0.9485623 ],
       [0.94709397, 0.94826593, 0.94690164],
       [0.94815201, 0.94735144, 0.94535901],
       [0.94306878, 0.95136905, 0.94647578],
       [0.9420071 , 0.94512757, 0.9491235 ],
       [0.94442999, 0.94380897, 0.94770702],
       [0.94481568, 0.94533773, 0.94499107],
       [0.94518973, 0.9434103 , 0.94545222],
       [0.94483024, 0.94358172, 0.9438695 ],
       [0.94343468, 0.93891095, 0.94696626],
       [0.94444851, 0.94286084, 0.93966613],
       [0.94388496, 0.94475607, 0.94025476],
       [0.94337488, 0.94197988, 0.93552924],
       [0.94537581, 0.94558828, 0.93831718],
       [0.93663487, 0.94449312, 0.94224777],
       [0.93934747, 0.94436833, 0.94116307],
       [0.94287471, 0.94322258, 0.94438604],
       [0.9430939 , 0.942834  , 0.94400359],
       [0.

In [429]:
y_train

,BOMBAY,CALI,SIRA
8,1,0,0
106,0,0,1
76,0,1,0
9,1,0,0
89,0,1,0
...,...,...,...
37,1,0,0
2,1,0,0
33,1,0,0
52,0,1,0


In [430]:
(np.argmax(y_test.to_numpy(),axis=1)==NetworkOutput).sum()/y_test.shape[0]

0.5

In [431]:
y_test.shape[0]

30

In [432]:
hidden_layers=3
numberOfNeuronsPerLayer=[2,3,3]
LR=0.01
epochs=10
bias=True
Layers_dict=[]


### Intializing the Hidden Layers